In [14]:
import requests
from bs4 import BeautifulSoup
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
import torch
import pandas as pd

In [15]:
# Load the FinBERT sentiment analysis model
tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')
model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert')

In [16]:
finbert = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [17]:

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                  ' Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
}

In [18]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

In [19]:
sp500_table = pd.read_html(url)[0]

In [20]:
sp500_table['Security'].tolist()

['3M',
 'A. O. Smith',
 'Abbott Laboratories',
 'AbbVie',
 'Accenture',
 'Adobe Inc.',
 'Advanced Micro Devices',
 'AES Corporation',
 'Aflac',
 'Agilent Technologies',
 'Air Products',
 'Airbnb',
 'Akamai Technologies',
 'Albemarle Corporation',
 'Alexandria Real Estate Equities',
 'Align Technology',
 'Allegion',
 'Alliant Energy',
 'Allstate',
 'Alphabet Inc. (Class A)',
 'Alphabet Inc. (Class C)',
 'Altria',
 'Amazon',
 'Amcor',
 'Ameren',
 'American Electric Power',
 'American Express',
 'American International Group',
 'American Tower',
 'American Water Works',
 'Ameriprise Financial',
 'Ametek',
 'Amgen',
 'Amphenol',
 'Analog Devices',
 'Ansys',
 'Aon',
 'APA Corporation',
 'Apple Inc.',
 'Applied Materials',
 'Aptiv',
 'Arch Capital Group',
 'Archer Daniels Midland',
 'Arista Networks',
 'Arthur J. Gallagher & Co.',
 'Assurant',
 'AT&T',
 'Atmos Energy',
 'Autodesk',
 'Automatic Data Processing',
 'AutoZone',
 'AvalonBay Communities',
 'Avery Dennison',
 'Axon Enterprise',
 'B

In [10]:
sp500_table = sp500_table[['Symbol', 'Security']]

import pandas as pd

# Load the S&P 500 table from the URL
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500_table = pd.read_html(url)[0]

# Create a dictionary with symbols as keys and company names as values
sp500_dict = dict(zip(sp500_table['Symbol'], sp500_table['Security']))

# Convert the dictionary to JSON and save it to a file
import json
with open('sp500_companies.json', 'w') as f:
    json.dump(sp500_dict, f, indent=4)

In [21]:
sp500_table = sp500_table['Symbol'].tolist()
sp500_table

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BK',
 'BBWI',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BA',
 'BKNG',
 'BWA',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'BLDR',
 'BG',
 'BXP',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CAG',
 'COP',
 'ED',

In [29]:
"BWA" in sp500_table
index = sp500_table.index("NSC")
print(index)

343


In [11]:
import requests
from bs4 import BeautifulSoup
import json

# URL for the S&P 500 companies from SlickCharts
url = 'https://www.slickcharts.com/sp500'

# Make a request to fetch the page content
response = requests.get(url,headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
# Try to find the table with the company data
table = soup.find('tbody')

if table:
    # Dictionary to store company symbol and name
    companies = {}

    # Iterate through table rows and extract company names and symbols
    for row in table.find_all('tr')[1:]:  # Skipping the header row
        columns = row.find_all('td')
        if len(columns) > 1:
            symbol = columns[2].get_text(strip=True)  # Get stock symbol
            name = columns[1].get_text(strip=True)    # Get company name
            companies[symbol] = name

    # Save the company data to a JSON file
    with open('sp500_companies.json', 'w') as json_file:
        json.dump(companies, json_file, indent=4)

    print("JSON file created successfully!")
    print(companies)
else:
    print("Error: Unable to find the table on the webpage.")


JSON file created successfully!
{'MSFT': 'Microsoft Corp', 'NVDA': 'Nvidia Corp', 'AMZN': 'Amazon.com Inc', 'META': 'Meta Platforms, Inc. Class A', 'GOOGL': 'Alphabet Inc. Class A', 'BRK.B': 'Berkshire Hathaway Class B', 'GOOG': 'Alphabet Inc. Class C', 'LLY': 'Eli Lilly & Co.', 'AVGO': 'Broadcom Inc.', 'TSLA': 'Tesla, Inc.', 'JPM': 'Jpmorgan Chase & Co.', 'UNH': 'Unitedhealth Group Incorporated', 'XOM': 'Exxon Mobil Corporation', 'V': 'Visa Inc.', 'PG': 'Procter & Gamble Company', 'MA': 'Mastercard Incorporated', 'COST': 'Costco Wholesale Corp', 'JNJ': 'Johnson & Johnson', 'HD': 'Home Depot, Inc.', 'ABBV': 'Abbvie Inc.', 'WMT': 'Walmart Inc.', 'NFLX': 'Netflix Inc', 'MRK': 'Merck & Co., Inc.', 'BAC': 'Bank of America Corporation', 'KO': 'Coca-Cola Company', 'ORCL': 'Oracle Corp', 'CRM': 'Salesforce, Inc.', 'AMD': 'Advanced Micro Devices', 'CVX': 'Chevron Corporation', 'PEP': 'Pepsico, Inc.', 'TMO': 'Thermo Fisher Scientific, Inc.', 'ADBE': 'Adobe Inc.', 'LIN': 'Linde Plc', 'MCD': "Mcd

In [12]:
sp500_table[322:]

['MRNA',
 'MHK',
 'MOH',
 'TAP',
 'MDLZ',
 'MPWR',
 'MNST',
 'MCO',
 'MS',
 'MOS',
 'MSI',
 'MSCI',
 'NDAQ',
 'NTAP',
 'NFLX',
 'NEM',
 'NWSA',
 'NWS',
 'NEE',
 'NKE',
 'NI',
 'NDSN',
 'NSC',
 'NTRS',
 'NOC',
 'NCLH',
 'NRG',
 'NUE',
 'NVDA',
 'NVR',
 'NXPI',
 'ORLY',
 'OXY',
 'ODFL',
 'OMC',
 'ON',
 'OKE',
 'ORCL',
 'OTIS',
 'PCAR',
 'PKG',
 'PANW',
 'PARA',
 'PH',
 'PAYX',
 'PAYC',
 'PYPL',
 'PNR',
 'PEP',
 'PFE',
 'PCG',
 'PM',
 'PSX',
 'PNW',
 'PNC',
 'POOL',
 'PPG',
 'PPL',
 'PFG',
 'PG',
 'PGR',
 'PLD',
 'PRU',
 'PEG',
 'PTC',
 'PSA',
 'PHM',
 'QRVO',
 'PWR',
 'QCOM',
 'DGX',
 'RL',
 'RJF',
 'RTX',
 'O',
 'REG',
 'REGN',
 'RF',
 'RSG',
 'RMD',
 'RVTY',
 'ROK',
 'ROL',
 'ROP',
 'ROST',
 'RCL',
 'SPGI',
 'CRM',
 'SBAC',
 'SLB',
 'STX',
 'SRE',
 'NOW',
 'SHW',
 'SPG',
 'SWKS',
 'SJM',
 'SW',
 'SNA',
 'SOLV',
 'SO',
 'LUV',
 'SWK',
 'SBUX',
 'STT',
 'STLD',
 'STE',
 'SYK',
 'SMCI',
 'SYF',
 'SNPS',
 'SYY',
 'TMUS',
 'TROW',
 'TTWO',
 'TPR',
 'TRGP',
 'TGT',
 'TEL',
 'TDY',
 'TFX',
 '

In [30]:
MAX_TOKENS = 512
from datetime import datetime
api_url = "http://127.0.0.1:8000/api/sentiment-analysis"
for stock in sp500_table:
    url = f"https://finance.yahoo.com/quote/{stock}/news?p={stock}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        news_items = soup.find_all('li', class_='stream-item')
        
        sentiment_scores = []
        print(f"\nScraped Headlines and Sentiment Scores for {stock}:")
        
        for i, item in enumerate(news_items):    
            headline_tag = item.find('h3')
            headline = headline_tag.get_text().strip() if headline_tag else ''
            
            description_tag = item.find('p')
            description = description_tag.get_text().strip() if description_tag else ''
            
            text = f"{headline} {description}"
            
            if len(text) > MAX_TOKENS:
                # If text is too long, use only the description
                text = description if len(description) <= MAX_TOKENS else description[:MAX_TOKENS]
            
            if description and headline:
                finbert_result = finbert(text)[0]
                
                sentiment_score = finbert_result['score']
                sentiment_label = finbert_result['label']
                
                if sentiment_label == 'positive' or sentiment_label == 'negative':
                    if sentiment_label == 'negative':
                        sentiment_score = -sentiment_score 
                    sentiment_scores.append(sentiment_score)
                
                print(f"{i+1}. Headline: {headline}")
                print(f"   Description: {description}")
                print(f"   Sentiment: {sentiment_label}, Score: {sentiment_score:.4f}")
                print('-' * 80)
                
        if sentiment_scores:
            average_sentiment = sum(sentiment_scores) / len(sentiment_scores)
            print(f"Average Sentiment Score for {stock}: {average_sentiment:.4f}")
            data = {
                "stock_symbol": stock,
                "sentiment_score": average_sentiment,  # Example sentiment score
                "analysis_date": datetime.now().strftime('%Y-%m-%d %H:%M:%S')  # Current date and time
            }
            response = requests.post(api_url, json=data)
            if response.status_code == 201:
                print("Data successfully sent to the API!")
                print(response.json())  # Print the response from the API
            else:
                try:
                    print(f"Failed to send data. Status code: {response.status_code}")
                    print(response.json())  # Print the error message
                except(e) :
                    print("error ",e)
        else:
            print("No sentiment scores to average.")

    else:
        print(f"Failed to fetch data for {stock}. Status code: {response.status_code}")


Scraped Headlines and Sentiment Scores for NTRS:
1. Headline: Northern Trust Decreases Prime Rate
   Description: CHICAGO, September 18, 2024--Northern Trust has decreased its prime rate from 8.50% to 8.00%, effective Thursday, September 19, 2024.
   Sentiment: negative, Score: -0.9564
--------------------------------------------------------------------------------
3. Headline: Bring Your Child to Work Day at Northern Trust India
   Description: NORTHAMPTON, MA / ACCESSWIRE / September 17, 2024 / Northern Trust Over 300 partners participated in our 2024 Bring Your Child to Work Day hosted by our employee-led Working Families Business Resource Council. It was truly an event filled with joy, ...
   Sentiment: neutral, Score: 0.6795
--------------------------------------------------------------------------------
4. Headline: 2Xideas Appoints Northern Trust for Integrated Trading Solutions
   Description: LUXEMBOURG, September 16, 2024--Northern Trust (Nasdaq: NTRS) is pleased to announce

In [32]:
MAX_TOKENS = 512
from datetime import datetime
api_url = "http://3.88.194.24/api/sentiment-analysis"
a = ['AAL','IFF','PG','NVDA','MSFT','INTC','TSLA','AMZN']
for stock in a:
    url = f"https://finance.yahoo.com/quote/{stock}/news?p={stock}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        news_items = soup.find_all('li', class_='stream-item')
        
        sentiment_scores = []
        print(f"\nScraped Headlines and Sentiment Scores for {stock}:")
        
        for i, item in enumerate(news_items):    
            headline_tag = item.find('h3')
            headline = headline_tag.get_text().strip() if headline_tag else ''
            
            description_tag = item.find('p')
            description = description_tag.get_text().strip() if description_tag else ''
            
            text = f"{headline} {description}"
            
            if len(text) > MAX_TOKENS:
                # If text is too long, use only the description
                text = description if len(description) <= MAX_TOKENS else description[:MAX_TOKENS]
            
            if description and headline:
                finbert_result = finbert(text)[0]
                
                sentiment_score = finbert_result['score']
                sentiment_label = finbert_result['label']
                
                if sentiment_label == 'positive' or sentiment_label == 'negative':
                    if sentiment_label == 'negative':
                        sentiment_score = -sentiment_score 
                    sentiment_scores.append(sentiment_score)
                
                print(f"{i+1}. Headline: {headline}")
                print(f"   Description: {description}")
                print(f"   Sentiment: {sentiment_label}, Score: {sentiment_score:.4f}")
                print('-' * 80)
                
        if sentiment_scores:
            average_sentiment = sum(sentiment_scores) / len(sentiment_scores)
            print(f"Average Sentiment Score for {stock}: {average_sentiment:.4f}")
            data = {
                "stock_symbol": stock,
                "sentiment_score": average_sentiment,  # Example sentiment score
                "analysis_date": datetime.now().strftime('%Y-%m-%d %H:%M:%S')  # Current date and time
            }
            response = requests.post(api_url, json=data)
            if response.status_code == 201:
                print("Data successfully sent to the API!")
                print(response.json())  # Print the response from the API
            else:
                try:
                    print(f"Failed to send data. Status code: {response.status_code}")
                    print(response.json())  # Print the error message
                except(e) :
                    print("error ",e)
        else:
            print("No sentiment scores to average.")

    else:
        print(f"Failed to fetch data for {stock}. Status code: {response.status_code}")


Scraped Headlines and Sentiment Scores for AAL:
1. Headline: Why Investors Need to Take Advantage of These 2 Transportation Stocks Now
   Description: Finding stocks expected to beat quarterly earnings estimates becomes an easier task with our Zacks Earnings ESP.
   Sentiment: neutral, Score: 0.8412
--------------------------------------------------------------------------------
3. Headline: Zacks.com featured highlights include American Airlines, Alaska Air Group, Warner Bros. Discovery, Dana and Best Buy
   Description: American Airlines, Alaska Air Group, Warner Bros. Discovery, Dana and Best Buy are part of the Zacks Screen of the Week article.
   Sentiment: neutral, Score: 0.9505
--------------------------------------------------------------------------------
4. Headline: Is American Airlines Group Inc. (AAL) Among the Worst Airline Stocks to Buy?
   Description: We recently compiled a list of the 10 Worst Airline Stocks To Buy According to Short Sellers. In this article, we are 